In [6]:
import pandas as pd
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="0"
#from scikeras.wrappers import KerasClassifier, KerasRegressor
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras import Sequential
from keras.layers import Dense, Dropout
import keras.metrics
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, TensorBoard
from MLscores import calc_metrics, metrics_dict, cmvals, recall, hybridrecall
import keras.backend as K
import tensorflow as tf
from functools import partial
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import recall_score, make_scorer
from xgboost import XGBClassifier
from check_and_prepare_dataset import load_dataset, prepare_dataset
from manage_model import create_NN_model, create_sklearn_model, allowgrowthgpus
import re
#import eli5
#from eli5.sklearn import PermutationImportance

In [51]:
#params={'ES_mindelta': 0.001, 'ES_monitor': 'loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 10}, 'dropout': 0.1, 'feature_drop': ('dir_max', 'dom_dir', 'month', 'wkd'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 70}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
#params={'ES_mindelta': 0.0001, 'ES_monitor': 'loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 5}, 'dropout': 0.3, 'feature_drop': ('dir_max', 'dom_dir', 'month', 'wkd'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 200.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
params = {}
#params['nn_auc']= {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 5}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'pop', 'corine_gr1', 'corine_gr2', 'corine_gr3', 'corine_gr4', 'corine_gr5', 'corine_gr6', 'corine_gr7', 'corine_gr8', 'corine_gr9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 700.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
params['nn_h1'] = {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 2}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'corine_gr1', 'corine_gr2', 'corine_gr3', 'corine_gr4', 'corine_gr5', 'corine_gr6', 'corine_gr7', 'corine_gr8', 'corine_gr9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 60.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
params['nn_h2'] = {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 5}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'pop', 'corine_gr1', 'corine_gr2', 'corine_gr3', 'corine_gr4', 'corine_gr5', 'corine_gr6', 'corine_gr7', 'corine_gr8', 'corine_gr9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 700.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
params['nn_nh2'] = {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 5}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'pop', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 1700.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
params['nn_nh5'] = {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 10}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'pop', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 1100.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}

In [52]:
models={}
for paramset in params:
    X, y, g=load_dataset('/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv', \
                         params[paramset]['feature_drop']+ \
                         ('evi','dom_vel', 'max_dew_temp', 'min_dew_temp', 'max_temp', 'min_temp')) #correlated features
    creatennmodel = partial(create_NN_model, params[paramset], X)
    models[paramset]={}
    models[paramset]['model'] = KerasClassifier(build_fn=creatennmodel, batch_size=params[paramset]['batch_size'], epochs=300, verbose=0,)
    models[paramset]['params'] = params[paramset]

Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv
before nan drop: 41771
after nan drop: 41771
after dup. drop: 41684
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['evi', 'bin_corine_gr31', 'bin_month_7', 'bin_weekday_4', 'bin_dom_dir_1', 'min_temp', 'bin_dom_dir_5', 'bin_dom_dir_4', 'bin_dir_max_6', 'bin_month_4', 'max_dew_temp', 'bin_corine_gr23', 'bin_dir_max_5', 'bin_corine_gr4', 'bin_month_10', 'bin_dir_max_3', 'min_dew_temp', 'bin_corine_gr22', 'bin_dom_dir_6', 'bin_corine_gr5', 'bin_weekday_7', 'bin_weekday_6', 'bin_dom_dir_8', 'dom_vel', 'bin_dir_max_4', 'bin_dom_dir_2', 'bin_corine_gr32', 'bin_corine_gr33', 'bin_weekday_1', 'bin_month_8', 'bin_dir_max_2', 'max_temp', 'bin_corine_gr21', 'bin_weekday_2', 'bin_weekday_5', 'bin_dom_dir_3', 'bin_month_3', 'bin_corine_gr1', 'bin_month_6', 'bin_dir_max_1', 'bin_month_9', 'bin_weekday

/tmp/ipykernel_1228178/3627430412.py:8: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  models[paramset]['model'] = KerasClassifier(build_fn=creatennmodel, batch_size=params[paramset]['batch_size'], epochs=300, verbose=0,)
/tmp/ipykernel_1228178/3627430412.py:8: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  models[paramset]['model'] = KerasClassifier(build_fn=creatennmodel, batch_size=params[paramset]['batch_size'], epochs=300, verbose=0,)


before nan drop: 41771
after nan drop: 41771
after dup. drop: 41684
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['evi', 'bin_corine_gr31', 'bin_month_7', 'pop', 'bin_weekday_4', 'bin_dom_dir_1', 'min_temp', 'bin_dom_dir_5', 'bin_dom_dir_4', 'bin_dir_max_6', 'bin_month_4', 'max_dew_temp', 'bin_corine_gr23', 'bin_dir_max_5', 'bin_corine_gr4', 'bin_month_10', 'bin_dir_max_3', 'min_dew_temp', 'bin_corine_gr22', 'bin_dom_dir_6', 'bin_corine_gr5', 'bin_weekday_7', 'bin_weekday_6', 'bin_dom_dir_8', 'dom_vel', 'bin_dir_max_4', 'bin_dom_dir_2', 'bin_corine_gr32', 'bin_corine_gr33', 'bin_weekday_1', 'bin_month_8', 'bin_dir_max_2', 'max_temp', 'bin_corine_gr21', 'bin_weekday_2', 'bin_weekday_5', 'bin_dom_dir_3', 'bin_month_3', 'bin_corine_gr1', 'bin_month_6', 'bin_dir_max_1', 'bin_month_9', 'bin_weekday_3', 'bin_dir_max_8', 'bin_dom_dir_7', 'bin_corine_gr24', 'bin_month_5', 'bi

/tmp/ipykernel_1228178/3627430412.py:8: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  models[paramset]['model'] = KerasClassifier(build_fn=creatennmodel, batch_size=params[paramset]['batch_size'], epochs=300, verbose=0,)
/tmp/ipykernel_1228178/3627430412.py:8: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  models[paramset]['model'] = KerasClassifier(build_fn=creatennmodel, batch_size=params[paramset]['batch_size'], epochs=300, verbose=0,)


In [53]:
paramssk={}
#RF best
paramssk['RF']={'algo': 'RF','bootstrap': True, 'class_weights': {0: 1, 1: 10}, 'criterion': 'entropy', 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'max_depth': 1000, 'max_features': 2.0, 'min_samples_leaf': 10, 'min_samples_split': 2, 'n_estimators': 350}
#ET best
paramssk['ET']={'algo': 'XT', 'bootstrap': True, 'class_weights': {0: 1, 1: 1}, 'criterion': 'entropy', 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'max_depth': None, 'max_features': 2.0, 'min_samples_leaf': 50, 'min_samples_split': 150, 'n_estimators': 800}
#XG
paramssk['XGB']={'algo': 'XGB', 'alpha': 60, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'gamma': 0.1, 'lambda': 21.0, 'max_depth': 44.0, 'n_estimators': 640, 'scale_pos_weight': 15, 'subsample': 0.6}
for skalgo in paramssk:
    print(paramssk[skalgo]['algo'])
    X, y, g=load_dataset('/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv', \
                         params[paramset]['feature_drop'] +
                        ('evi','dom_vel', 'max_dew_temp', 'min_dew_temp', 'max_temp', 'min_temp'))
    models[skalgo]={}
    models[skalgo]['model'] = create_sklearn_model(paramssk[skalgo],X)
    models[skalgo]['params']=paramssk[skalgo]


RF
Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv
before nan drop: 41771
after nan drop: 41771
after dup. drop: 41684
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['bin_corine_8', 'bin_corine_35', 'bin_corine_9', 'bin_month_7', 'bin_weekday_4', 'bin_dom_dir_1', 'min_temp', 'bin_corine_1', 'bin_dir_max_6', 'bin_month_4', 'bin_corine_42', 'bin_dir_max_5', 'bin_corine_7', 'bin_corine_30', 'bin_corine_10', 'bin_dir_max_3', 'bin_corine_22', 'bin_corine_5', 'bin_corine_17', 'bin_corine_18', 'bin_corine_37', 'bin_corine_32', 'bin_corine_16', 'bin_weekday_6', 'bin_corine_11', 'dom_vel', 'bin_corine_13', 'bin_dom_dir_8', 'bin_corine_33', 'bin_dir_max_4', 'bin_corine_25', 'bin_corine_2', 'bin_month_8', 'bin_dir_max_2', 'max_temp', 'bin_weekday_2', 'bin_weekday_5', 'bin_corine_24', 'bin_month_3', 'bin_dom_dir_3', 'bin_corine_21', 'bin_month_6'

In [67]:
models

{'nn_auc': {'model': <keras.wrappers.scikit_learn.KerasClassifier at 0x7f1c38446f20>,
  'params': {'ES_mindelta': 0.002,
   'ES_monitor': 'val_loss',
   'ES_patience': 10,
   'batch_size': 512,
   'class_weights': {0: 1, 1: 5},
   'dropout': None,
   'feature_drop': ('month',
    'weekday',
    'dom_dir',
    'dir_max',
    'pop',
    'corine_gr1',
    'corine_gr2',
    'corine_gr3',
    'corine_gr4',
    'corine_gr5',
    'corine_gr6',
    'corine_gr7',
    'corine_gr8',
    'corine_gr9'),
   'max_epochs': 2000,
   'metric': 'accuracy',
   'n_internal_layers': (0, {'layer_1_0_nodes': 700.0}),
   'optimizer': {'adam_params': None, 'name': 'Adam'}}},
 'nn_h1': {'model': <keras.wrappers.scikit_learn.KerasClassifier at 0x7f1b304450f0>,
  'params': {'ES_mindelta': 0.002,
   'ES_monitor': 'val_loss',
   'ES_patience': 10,
   'batch_size': 512,
   'class_weights': {0: 1, 1: 2},
   'dropout': None,
   'feature_drop': ('month',
    'weekday',
    'dom_dir',
    'dir_max',
    'corine_gr1',
   

In [54]:
#models1={'RF':models['RF'], 'ET':models['ET']}
allowgrowthgpus()
for modname in models:
    X, y, g=load_dataset('/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv', \
                         models[modname]['params']['feature_drop']+ \
                         ('evi','dom_vel', 'max_dew_temp', 'min_dew_temp', 'max_temp', 'min_temp'))
    print(modname)
    models[modname]['model'].fit(X,y)
    break

2023-05-16 12:54:23.440072: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-16 12:54:23.457325: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-16 12:54:23.457413: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-16 12:54:23.458629: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other 

1 Physical GPUs, 1 Logical GPUs
Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv
before nan drop: 41771
after nan drop: 41771
after dup. drop: 41684
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['evi', 'bin_corine_gr31', 'bin_month_7', 'bin_weekday_4', 'bin_dom_dir_1', 'min_temp', 'bin_dom_dir_5', 'bin_dom_dir_4', 'bin_dir_max_6', 'bin_month_4', 'max_dew_temp', 'bin_corine_gr23', 'bin_dir_max_5', 'bin_corine_gr4', 'bin_month_10', 'bin_dir_max_3', 'min_dew_temp', 'bin_corine_gr22', 'bin_dom_dir_6', 'bin_corine_gr5', 'bin_weekday_7', 'bin_weekday_6', 'bin_dom_dir_8', 'dom_vel', 'bin_dir_max_4', 'bin_dom_dir_2', 'bin_corine_gr32', 'bin_corine_gr33', 'bin_weekday_1', 'bin_month_8', 'bin_dir_max_2', 'max_temp', 'bin_corine_gr21', 'bin_weekday_2', 'bin_weekday_5', 'bin_dom_dir_3', 'bin_month_3', 'bin_corine_gr1', 'bin_month_6', 'bin_dir_max

2023-05-16 12:54:24.858303: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [72]:
ypred=models['nn_h1']['model'].predict(X)

1303/1303 [==============================] - 0s 374us/step


In [82]:
from sklearn.metrics import classification_report
print(classification_report(y,ypred))

              precision    recall  f1-score   support

         0.0       0.92      0.91      0.92     27814
         1.0       0.83      0.84      0.83     13870

    accuracy                           0.89     41684
   macro avg       0.87      0.87      0.87     41684
weighted avg       0.89      0.89      0.89     41684



In [83]:
recall_score(y,ypred,pos_label=1)

0.8364095169430426

In [55]:
rec1_scorer = make_scorer(recall_score, pos_label=1)
rec0_scorer = make_scorer(recall_score, pos_label=0)
scoring={'recall1':rec1_scorer, 'recall0':rec0_scorer}
modelsresults=[]
for modname in models:
    print(modname)
    #models[modname].fit(X,y)
    #ypred=models['nn_auc'].predict(X)
    X, y, g=load_dataset('/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv', \
                          models[modname]['params']['feature_drop']+ \
                         ('evi','dom_vel', 'max_dew_temp', 'min_dew_temp', 'max_temp', 'min_temp'))
    r=permutation_importance(models[modname]['model'], X, y, n_repeats=3, scoring=scoring)
    modelsresults+=[r]
    break

nn_h1
Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv
before nan drop: 41771
after nan drop: 41771
after dup. drop: 41684
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['evi', 'bin_corine_gr31', 'bin_month_7', 'bin_weekday_4', 'bin_dom_dir_1', 'min_temp', 'bin_dom_dir_5', 'bin_dom_dir_4', 'bin_dir_max_6', 'bin_month_4', 'max_dew_temp', 'bin_corine_gr23', 'bin_dir_max_5', 'bin_corine_gr4', 'bin_month_10', 'bin_dir_max_3', 'min_dew_temp', 'bin_corine_gr22', 'bin_dom_dir_6', 'bin_corine_gr5', 'bin_weekday_7', 'bin_weekday_6', 'bin_dom_dir_8', 'dom_vel', 'bin_dir_max_4', 'bin_dom_dir_2', 'bin_corine_gr32', 'bin_corine_gr33', 'bin_weekday_1', 'bin_month_8', 'bin_dir_max_2', 'max_temp', 'bin_corine_gr21', 'bin_weekday_2', 'bin_weekday_5', 'bin_dom_dir_3', 'bin_month_3', 'bin_corine_gr1', 'bin_month_6', 'bin_dir_max_1', 'bin_month_9', 'bin_w

1303/1303 [==============================] - 0s 324us/step


In [65]:
def getcorinedesc(corinefeat):
    r=re.search(r"\d+$",corinefeat)
    corid=r.group(0)
    corstr=dfcorcats.loc[dfcorcats['Unnamed: 3']==corid,'CORINE LAND COVER LEGEND'].values[0]
    return corstr

def colcond1(X,i):
    return not any(c in X.columns[i] for c in ['dir_max','dom_dir','wkd','month','corine'])


def colcond2(X,i):
    return 'corine' in X.columns[i]
    
#for r in modelsresults
def resdict(condfunc):
    results_lists={}
    for s in scoring:
        results_lists['%s %s'%(modname,s)]=[]
        cnt=0
        for i in r[s].importances_mean.argsort()[::-1]:
            #if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
            cnt+=1
            if condfunc(X,i):
                results_dict={}
                results_dict['place %s %s'%(modname,s)]=cnt
                results_dict['rank %s %s'%(modname,s)]=X.columns[i]
                results_dict['perm. imp. %s %s'%(modname,s)]=r[s].importances_mean[i]
                results_lists['%s %s'%(modname,s)]+=[results_dict]
    return results_lists

In [15]:
#for i in r.importances_mean.argsort()[::-1]:
    #if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
#    print(f"{X.columns[i]:<17}"
#        f"{r.importances_mean[i]:.3f}"
#        f" +/- {r.importances_std[i]:.3f}")

In [56]:
results_lists=resdict(colcond1)
results_pd=[]
for mod in results_lists:
    results_pd+=[pd.DataFrame(results_lists[mod])]
all_res_pd=pd.concat(results_pd, axis=1)

In [84]:
all_res_pd

,place nn_h1 recall1,rank nn_h1 recall1,perm. imp. nn_h1 recall1,place nn_h1 recall0,rank nn_h1 recall0,perm. imp. nn_h1 recall0
0,1,res_max,0.243547,2,ypos,0.033077
1,3,mean_temp,0.109132,4,mean_dew_temp,0.030177
2,5,f81,0.100577,5,res_max,0.029937
3,7,dem,0.079620,7,xpos,0.017210
4,9,ndvi,0.050012,17,dem,0.006424
5,12,xpos,0.041000,19,f81,0.005788
6,13,rain_7_days,0.040759,20,frequency,0.004866
7,18,mean_dew_temp,0.025210,21,lst_night,0.004410
8,21,ypos,0.016679,24,pop,0.001210
9,22,road_dens,0.015237,26,road_dens,0.001162


In [58]:
results_lists=resdict(colcond2)
results_pd=[]
for mod in results_lists:
    results_pd+=[pd.DataFrame(results_lists[mod])]
all_res_pd2=pd.concat(results_pd, axis=1)
all_res_pd2['rank nn_h1 recall0']=all_res_pd2['rank nn_h1 recall0'].apply(getcorinedesc)
all_res_pd2['rank nn_h1 recall1']=all_res_pd2['rank nn_h1 recall1'].apply(getcorinedesc)

In [70]:
all_res_pd2

,place nn_h1 recall1,rank nn_h1 recall1,perm. imp. nn_h1 recall1,place nn_h1 recall0,rank nn_h1 recall0,perm. imp. nn_h1 recall0
0,2,3.2.3 Sclerophyllous vegetation,0.184331,1,3.2.3 Sclerophyllous vegetation,0.044270
1,4,3.2.1 Natural grasslands,0.106777,3,3.2.1 Natural grasslands,0.030308
2,6,3.2.4 Transitional woodland-shrub,0.094833,6,3.2.4 Transitional woodland-shrub,0.023094
3,8,2.4.3 Land principally occupied by agriculture...,0.066643,8,3.3.4 Burnt areas,0.016574
4,10,3.3.4 Burnt areas,0.048354,9,2.4.3 Land principally occupied by agriculture...,0.012883
5,11,2.2.3 Olive groves,0.044773,10,3.1.3 Mixed forest,0.012512
6,14,2.4.2 Complex cultivation patterns,0.036866,11,3.1.2 Coniferous forest,0.012176
7,15,2.1.1 Non-irrigated arable land,0.035184,12,3.1.1 Broad-leaved forest,0.011014
8,16,3.3.3 Sparsely vegetated areas,0.031435,13,3.3.3 Sparsely vegetated areas,0.010834
9,17,3.1.2 Coniferous forest,0.026196,14,2.1.1 Non-irrigated arable land,0.009923


In [62]:
all_res_pd2.to_csv('/mnt/nvme2tb/ffp/results/newdefCV/permimpcorine.csv', index=False)

In [63]:
all_res_pd.to_csv('/mnt/nvme2tb/ffp/results/newdefCV/permimpmainfeat.csv', index=False)